## Ensembles of Decision Trees

*Ensembles* are methods that combines multiple machine learning models to create more powerful models. There are several machine learning models that belong to this category, but there are two ensemble models that have proven to be effeictive on a wide range of datasets for classification and regression. Both of them use decision trees as their building blocks: **Random forest** and **Gradient boosted** decision trees.

### Random forests

As mentioned before, a main drawback of decision trees is that they tend to overfit the training data. Random forests are one way to address this problem which is a collection of decision trees where each tree is slightly different from each others. 
The idea behind random forests is the each tree might do a relatively good job of predicting, but will likely overfit on part of the data. If we build many trees, all of which work well and overfit in different ways, we can reduce the amount of overfitting by averaging their results. 
Random forests get their name from injecting randomness into the tree building to ensure each tree is different. There are two ways in which the tree in a random forests are randomized: 
- By selecting the data points used to build a tree 
- By selecting the features in each split test. 


#### Building random forests: